# Land Surface Temperature  (LST)


Fuentes a consultar: 

https://medium.com/@Riski-Saputra/mapping-land-surface-temperature-lst-using-landsat-8-imagery-in-arcmap-99bd64f2c3fa


Leer este comentario: 

https://gis.stackexchange.com/questions/433305/error-calculating-lst-in-qgis


**Capitulo 36) Heat Islands**

**IMPORTACION DE LIBRERIAS**

In [3]:
import ee
import geemap
import os 

## 1) Autentificación 

In [4]:
# 1) Obtención de la dirección de trabajo 
direction = os.getcwd()

# Remove the last part of the path
direction = os.path.dirname(direction)

# Dirección de la llave 
service_account = direction + '/conf/local/gcp-for-data-science-397913-4fd843feede1.json'

# Autentificación 
credentials = ee.ServiceAccountCredentials(email=None, key_file=service_account)
ee.Initialize(credentials)

## 2) Importación de imagenes 

## 3) Procesamiento 
Conceptos relevantes:

**SENSOR A UTILIZAR:**  En este caso utulizaremos landsat 9 debido a su sensor infrarrojo termico (Thermal Infrared Sensor (TIRS))







In [ ]:
### AQUI VOY !!!!, hay que hacer bien la descripción del procesamiento que estoy haciendo en la parte de arriba. 

In [5]:
# Define Area of Interest (AOI)
aoi = ee.Geometry.Point([-99.1332, 19.4326])  # Example: Mexico City

# Load Landsat 9 Surface Reflectance Collection
landsat9 = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2') \
             .filterBounds(aoi) \
             .filterDate('2023-01-01', '2023-12-31') \
             .filterMetadata('CLOUD_COVER', 'less_than', 10)

In [6]:
# Select the most recent image
image = landsat9.first()

In [7]:
# Visualize the True Color Composite
true_color = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
    'min': 0,
    'max': 30000,
    'gamma': 1.4
}

# Apply Scaling Factors (Required for L2 products)
def scale_image(image):
    sr = image.select('SR_B.*').multiply(0.0000275).add(-0.2)  # Surface Reflectance
    tir = image.select('ST_B10').multiply(0.00341802).add(149.0)  # Thermal Band 10
    return image.addBands(sr, overwrite=True).addBands(tir, overwrite=True)

image = scale_image(image)

# Calculate NDVI (for Emissivity Estimation)
ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')

# Estimate Land Surface Emissivity (LSE) using NDVI
def calculate_emissivity(ndvi):
    return ndvi.expression(
        '0.004 * NDVI + 0.986', {
            'NDVI': ndvi
        }).rename('Emissivity')

emissivity = calculate_emissivity(ndvi)

# Calculate Land Surface Temperature (LST) in Kelvin
lst = image.select('ST_B10').divide(emissivity).rename('LST')

# Convert LST to Celsius
lst_celsius = lst.subtract(273.15).rename('LST_Celsius')

# Visualization Parameters for LST
lst_vis = {
    'min': 20,
    'max': 40,
    'palette': ['blue', 'cyan', 'green', 'yellow', 'red']
}

# Display on Map
Map = geemap.Map()
Map.centerObject(aoi, 10)
Map.addLayer(image, true_color, "Landsat 9 True Color")
Map.addLayer(lst_celsius, lst_vis, "Land Surface Temperature (°C)")
Map

Map(center=[19.432599999999997, -99.1332], controls=(WidgetControl(options=['position', 'transparent_bg'], wid…